In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from sklearn.model_selection import train_test_split
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from math import floor
from time import time
import numpy as np
import sklearn.metrics as metrics
import keras.utils.np_utils as nputil
from keras.preprocessing.image import ImageDataGenerator


#load CIfAR-10 Dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

def pad(x, pad=4):
    return np.pad(x, ((0,0), (pad,pad),(pad,pad),(0,0)), mode='reflect')

def random_crop(x, random_crop_size = (32,32), sync_seed=None):
    np.random.seed(sync_seed)
    w, h = x.shape[1], x.shape[2]
    rangew = (w - random_crop_size[0]) // 2
    rangeh = (h - random_crop_size[1]) // 2
    offsetw = 0 if rangew == 0 else np.random.randint(rangew)
    offseth = 0 if rangeh == 0 else np.random.randint(rangeh)
    return x[:, offsetw:offsetw+random_crop_size[0], offseth:offseth+random_crop_size[1]]

x_train = pad(x_train)
x_test = pad(x_test)

x_train = x_train.astype('float32')
x_train = (x_train - [125.3, 123.0, 113.9]) / [63.0, 62.1, 66.7]
x_test = x_test.astype('float32')
x_test = (x_test - [125.3, 123.0, 113.9]) / [63.0, 62.1, 66.7]
y_train = nputil.to_categorical(y_train)
y_test = nputil.to_categorical(y_test)


generator = ImageDataGenerator(zca_epsilon=0,
                               horizontal_flip=True,
                               fill_mode='reflect',)

generator.fit(x_train, seed=0, augment=True)

test_generator = ImageDataGenerator(zca_epsilon=0,
                               horizontal_flip=True,
                               fill_mode='reflect',)

test_generator.fit(x_test, seed=0, augment=True)

def densenet(filters=16):
  growth_rate=12
  compression=0.5
  nr_blocks = 3
  d_rate = 0.2
  repetitions = 6, 12, 24, 16
  
  #composition fuction
  def composition(inp, filters, kernel_size=(3,3), d_rate=0.0):
    x = BatchNormalization()(inp)
    x = ReLU()(x)
    x = Conv2D(filters, 1, strides=1, padding='same')(x)
    out = Dropout(d_rate)(x)
    return out
  
  
  def dense_block(inputs, nr_layers = 4, filters = 16, d_rate = 0, bottleneck = True):
    for _ in range(nr_layers):
      if bottleneck:
        x = composition(inputs, 4*filters)
      else:
        x = inputs
      x = composition(x, filters, (3,3),d_rate)
      inputs = Concatenate(axis = -1)([x, inputs])
    return inputs
  

  def transition_block(inputs, filters, compression = 0.5,d_rate = 0.0):
    filters = floor(filters*compression)
    x = composition(inputs, filters, (1,1), d_rate)
    out = AvgPool2D((2,2), strides=(2,2), padding='same')(x)
    return out
  
  
  input = tf.keras.layers.Input(shape=(32,32,3))
  x = Conv2D(filters, (3,3), use_bias = False)(input)
  x = MaxPool2D((3,3), strides=(2,2), padding='same')(x)
  
  for r in repetitions:
    d = dense_block(x, growth_rate, r, d_rate, False)
    x = transition_block(d, growth_rate, compression, d_rate)
  
  x = GlobalAvgPool2D()(d)
  output = tf.keras.layers.Dense(10,activation='softmax')(x)
  model = Model(input, output)
  return model


model = densenet()
model.compile(optimizer=('adam'),loss='categorical_crossentropy',metrics=['acc'])
model.summary()
#model.fit(x_train, y_train,
     #        validation_data=(x_test, y_test),
      #       epochs=1,
       #      batch_size=32)

for i in range(0, 4):
    print('i = ' + str(i+1))
    for x_batch, y_batch in generator.flow(x_train, y_train, batch_size=len(x_train)):
        for testx_batch, testy_batch in test_generator.flow(x_test, y_test, batch_size=len(x_test)):
            x_batch = random_crop(x_batch)
            testx_batch = random_crop(testx_batch)
            model.fit(x_batch, y_batch, epochs=1, validation_data=(testx_batch, testy_batch),  batch_size=32)
            break
        break
yPreds = model.predict(random_crop(x_test))
yPred = np.argmax(yPreds, axis=1)
yPred = nputil.to_categorical(yPred)
yTrue = y_test

accuracy = metrics.accuracy_score(yTrue, yPred) * 100
print("Accuracy : ", accuracy)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 30, 30, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 15, 15, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 15, 15, 16)   64          max_pooling2d[0][0]              
______________________________________________________________________________________________

I have imported all the libraries, spillitting the data and By watiching the tutorial and reading the pdf, I have defined the functions for each part and predict the data. 